In [49]:
import numpy as np
import pandas as pd
import random
import math
import time
from shapely.geometry import Polygon, MultiPolygon, Point, box
from shapely.affinity import scale, translate
from shapely import wkt
from shapely.ops import transform
from pyproj import Transformer
import folium
from folium.features import DivIcon
from IPython.display import display, clear_output
from tqdm.notebook import tqdm
import ipywidgets
output = ipywidgets.Output()

In [2]:
random.seed(777)

## Helper Functions

In [3]:
def calculateArea(poly_wkt_str):
    polygon = wkt.loads(poly_wkt_str)
    area_sq_degrees = polygon.area
    centroid = polygon.centroid
    lon, lat = centroid.x, centroid.y
    utm_zone = int((lon + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    # # Set up the transformation
    transformer = Transformer.from_crs("EPSG:4326", proj_string, always_xy=True)
    utm_polygon = transform(transformer.transform, polygon)
    area_sq_meters = utm_polygon.area
    area_sq_kilometers = area_sq_meters / 1_000_000  # Convert to square kilometers
    return float(format(area_sq_kilometers, '.2f'))

In [28]:
def displayOSM(i, polygon, bbox, area_percentage=None, uniform=None):
    # Create a map with the center based on the bounding box
    map_center = [(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2]
    m = folium.Map(location=map_center, zoom_start=10) # SWEDEN ZOOM 4
    
    if isinstance(polygon, str):
        # Assuming the string is WKT format
        try:
            shape = wkt.loads(polygon)
            geo_json = shape.__geo_interface__
        except Exception as e:
            print(f"Error parsing polygon {e}")
    else:
        # Assuming it's already in a format folium can use
        geo_json = polygon
        
        # Add the polygon to the map with a unique style
    folium.GeoJson(
        geo_json,
        style_function=lambda x, color=f'#4400ff': {
                'fillColor': color,
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5',
                'fillOpacity': 0.5,
        }
    ).add_to(m)

    north, south, east, west = bbox[3], bbox[1], bbox[0], bbox[2]
    # Add the bounding box to the map
    folium.Rectangle(bounds=[[north, west], [south, east]], color="red", fill=False).add_to(m)

    folium.map.Marker(
        [north-0.25, east-0.1],
        icon=DivIcon(
            icon_size=(550,56),
            icon_anchor=(0,0),
            html='<div style="font-size: 18pt; color: #4400ff;">points '+str(i)+' | area '+str(area_percentage)+' km² | uniformity '+str(uniform)+'</div>',
            )
        ).add_to(m)
    display(m)
    clear_output(wait=True)
    time.sleep(0.5)  # Pause to see the update
    # return m

## Generator Functions

In [20]:
def genPolygon_AreaFIXED(INPUT_BBOX, num_points=None, area_percentage=None, circum_radius_factor=None):
    """
    Generate different polygons within the INPUT_BBOX in EPSG:4326 - WGS 84
    Ranges from small area polygon to large polygons with large area in OGC WKT format
    
    Args:
        INPUT_BBOX: List [min_lon, min_lat, max_lon, max_lat]
        area_percentage: Optional float between 0 and 1, representing what percentage of the bbox area 
                        the polygon should occupy. If None, a random value is used.
    Returns:
        String: WKT format polygon
    """
    min_lon, min_lat, max_lon, max_lat = INPUT_BBOX
          
    # Center of bbox
    center_lon = (min_lon + max_lon) / 2
    center_lat = (min_lat + max_lat) / 2

    # Anywhere of bbox
    # center_lon = random.uniform(min_lon, max_lon)
    # center_lat = random.uniform(min_lat, max_lat)

    # Get bbox dimensions
    bbox_width = max_lon - min_lon
    bbox_height = max_lat - min_lat
    # Calculate polygon dimensions based on area percentage For simplicity, we'll create a rectangular polygon
    scale_factor = math.sqrt(area_percentage)
    poly_width = bbox_width * scale_factor
    poly_height = bbox_height * scale_factor
    
    # Calculate polygon corners
    half_width = poly_width / 2
    half_height = poly_height / 2
    
    # Create a more interesting polygon by adding some random points and perturbations
    points = []
    
    for i in range(num_points):
        angle = 2 * math.pi * i / num_points
        
        # Base radius is determined by the desired area
        x_radius = half_width
        y_radius = half_height
        
        # Add some randomness but maintain approximate area
        radius_factor = random.uniform(0.5, 1.0) # How uniform are the points (0.1 = random,  1.0 = uniform)
        
        x = center_lon + math.cos(angle) * x_radius * radius_factor
        y = center_lat + math.sin(angle) * y_radius * radius_factor
        
        # Ensure points are within the bbox
        x = max(min(x, max_lon), min_lon)
        y = max(min(y, max_lat), min_lat)
        
        points.append((x, y))
    
    # Close the polygon by adding the first point at the end
    points.append(points[0])
    
    # Convert to WKT format
    wkt = "POLYGON (("
    wkt += ", ".join([f"{p[0]} {p[1]}" for p in points])
    wkt += "))"

    # Double-check OGC compliance
    # is_valid, message = validate_ogc_polygon(wkt)
    # if not is_valid:
    #     # Try to fix the polygon
    #     wkt = fix_ogc_polygon(wkt)
    return wkt

### Gen 1

In [21]:
def genPolygonExp(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR):
    """
    Generate a polygon with specified area in square kilometers
    
    Parameters:
    INPUT_BBOX: WKT string representing the bounding box
    NUM_OF_POINTS: Number of points in the polygon
    AREA_IN_km²: Target area in square kilometers
    UNIFORM_FACTOR: Controls shape regularity (0.0: very irregular, 1.0: perfectly regular)
    
    Returns:
    WKT string representing the generated polygon in EPSG:4326
    """
    # Parse the bounding box
    if type(INPUT_BBOX) == str:
        bbox = wkt.loads(INPUT_BBOX)
        bbox_bounds = bbox.bounds
        min_x, min_y, max_x, max_y = bbox_bounds
    else:
        min_x, min_y, max_x, max_y = INPUT_BBOX
        bbox = wkt.loads((box(*INPUT_BBOX, ccw=True)).wkt)
    
    # Compute the centroid for UTM zone determination
    centroid_x = (min_x + max_x) / 2
    centroid_y = (min_y + max_y) / 2
    
    # Determine UTM zone
    utm_zone = int((centroid_x + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    # Create transformers for WGS84 <-> UTM conversion
    transformer_to_utm = Transformer.from_crs("EPSG:4326", proj_string, always_xy=True)
    transformer_from_utm = Transformer.from_crs(proj_string, "EPSG:4326", always_xy=True)
    
    # Transform the bounding box to UTM
    utm_bbox = transform(transformer_to_utm.transform, bbox)
    utm_bounds = utm_bbox.bounds
    utm_min_x, utm_min_y, utm_max_x, utm_max_y = utm_bounds
    
    # Calculate the center point of the bounding box in UTM
    center_x = (utm_min_x + utm_max_x) / 2
    center_y = (utm_min_y + utm_max_y) / 2
    
    # Calculate the radius needed for the target area
    # For a regular polygon with n sides, area = (n/4) * r² * sin(2π/n)
    # Solving for r: r = sqrt(area / ((n/4) * sin(2π/n)))
    target_area_m2 = AREA_IN_km * 1_000_000  # Convert km² to m²
    
    # For a regular polygon, calculate the radius
    n = NUM_OF_POINTS
    regular_radius = math.sqrt(target_area_m2 / ((n/4) * math.sin(2 * math.pi / n)))
    
    # Generate points for the polygon
    angles = np.linspace(0, 2 * np.pi, NUM_OF_POINTS, endpoint=False)
    
    utm_points = []
    for angle in angles:
        # Start with perfectly regular distance (radius)
        base_distance = regular_radius
        
        # Add randomness based on the uniformity factor
        if UNIFORM_FACTOR < 1.0:
            # The lower the uniformity factor, the more randomness
            random_variation = (1.0 - UNIFORM_FACTOR) * random.uniform(0.5, 1.5)
            distance = base_distance * (1.0 + (1.0 - UNIFORM_FACTOR) * (random_variation - 1.0))
        else:
            distance = base_distance
        
        # Calculate point coordinates
        x = center_x + distance * np.cos(angle)
        y = center_y + distance * np.sin(angle)
        utm_points.append((x, y))
    
    # Create the polygon in UTM
    utm_polygon = Polygon(utm_points)
    
    # Calculate the area of the generated polygon
    initial_area_m2 = utm_polygon.area
    
    # Scale the polygon to match the target area
    scaling_factor = math.sqrt(target_area_m2 / initial_area_m2)
    
    # Apply scaling: translate to origin, scale, translate back
    def scale_around_center(x, y):
        dx = x - center_x
        dy = y - center_y
        return center_x + dx * scaling_factor, center_y + dy * scaling_factor
    
    scaled_utm_polygon = transform(lambda x, y: scale_around_center(x, y), utm_polygon)
    
    # Transform the UTM polygon back to WGS84 (EPSG:4326)
    wgs84_polygon = transform(transformer_from_utm.transform, scaled_utm_polygon)
    
    # Return the WKT representation
    return wgs84_polygon.wkt

## USER DEFINED GLOBAL VARIABLES

- https://observablehq.com/@jake-low/bounding-box-to-geojson-polygon

In [22]:
# INPUT_BBOX = [11.015629, 55.128649, 24.199222, 69.380313] ## Sweden
INPUT_BBOX = [11.360694444453532, 48.06152777781623, 11.723194444453823, 48.24819444448305] # MUNICH
INPUT_BBOX_WKT_STRING = (box(*INPUT_BBOX, ccw=True)).wkt
INPUT_BBOX_WKT_STRING

'POLYGON ((11.723194444453823 48.06152777781623, 11.723194444453823 48.24819444448305, 11.360694444453532 48.24819444448305, 11.360694444453532 48.06152777781623, 11.723194444453823 48.06152777781623))'

In [23]:
calculateArea(INPUT_BBOX_WKT_STRING) 

559.85

## BM 1 - POLYGON WITH FIXED AREA and VARYING NUMBER OF POINTS

In [41]:
NUM_OF_POINTS = 20
AREA_IN_km = 100 
UNIFORM_FACTOR = 0.5

for POINTS in range(3, NUM_OF_POINTS, 1):
    polygon = genPolygonExp(INPUT_BBOX, POINTS, AREA_IN_km, UNIFORM_FACTOR)
    # print(f"Polygon Points: {POINTS} \n\n{polygon}\n")
    displayOSM(POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

In [12]:
point_values = list(range(3, NUM_OF_POINTS, 10))  # 10, 20, 30, ... 100
point_values

[3, 13, 23, 33, 43]

In [15]:
for point in range(3, NUM_OF_POINTS, 10):
    print(point)

3
13
23
33
43


## BM 2 - POLYGON WITH FIXED NUMBER OF POINTS and VARYING AREA 

In [69]:
NUM_OF_POINTS = 3
UNIFORM_FACTOR = 0.4

for AREA_IN_km in np.arange(1, calculateArea(INPUT_BBOX_WKT_STRING), 50):
    polygon = genPolygonExp(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR)
    # print(f"Polygon Area: {calculateArea(polygon):.2f}km² \n\n{polygon}\n")
    displayOSM(NUM_OF_POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

## BM 3 - POLYGON WITH SINGLE HOLE FIXED AREA and VARYING NUMBER OF POINTS

### Generator

In [12]:
def genPolygonWithHoles(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR, NUMBER_OF_DISJOINT_HOLES):
    """
    Generate a polygon with specified area in square kilometers and containing disjoint holes
    
    Parameters:
    INPUT_BBOX: WKT string representing the bounding box
    NUM_OF_POINTS: Number of points in the outer polygon
    AREA_IN_km: Target area in square kilometers for the entire polygon (including holes)
    UNIFORM_FACTOR: Controls shape regularity (0.0: very irregular, 1.0: perfectly regular)
    NUMBER_OF_DISJOINT_HOLES: Number of holes to create within the polygon
    
    Returns:
    WKT string representing the generated polygon with holes in EPSG:4326
    """
    # Parse the bounding box
    if type(INPUT_BBOX) == str:
        bbox = wkt.loads(INPUT_BBOX)
        bbox_bounds = bbox.bounds
        min_x, min_y, max_x, max_y = bbox_bounds
    else:
        min_x, min_y, max_x, max_y = INPUT_BBOX
        bbox = wkt.loads((box(*INPUT_BBOX, ccw=True)).wkt)
    
    # Compute the centroid for UTM zone determination
    centroid_x = (min_x + max_x) / 2
    centroid_y = (min_y + max_y) / 2
    
    # Determine UTM zone
    utm_zone = int((centroid_x + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    # Create transformers for WGS84 <-> UTM conversion
    transformer_to_utm = Transformer.from_crs("EPSG:4326", proj_string, always_xy=True)
    transformer_from_utm = Transformer.from_crs(proj_string, "EPSG:4326", always_xy=True)
    
    # Transform the bounding box to UTM
    utm_bbox = transform(transformer_to_utm.transform, bbox)
    utm_bounds = utm_bbox.bounds
    utm_min_x, utm_min_y, utm_max_x, utm_max_y = utm_bounds
    
    # Calculate the center point of the bounding box in UTM
    center_x = (utm_min_x + utm_max_x) / 2
    center_y = (utm_min_y + utm_max_y) / 2
    
    # Define how much of the total area will be allocated to holes (30% for holes)
    hole_area_percentage = 0.3
    solid_area_percentage = 1.0 - hole_area_percentage
    
    # Calculate total area needed for the target visible area
    target_area_m2 = AREA_IN_km * 1_000_000  # Convert km² to m²
    total_area_needed = target_area_m2 / solid_area_percentage
    
    # For a regular polygon, calculate the radius
    n = NUM_OF_POINTS
    regular_radius = math.sqrt(total_area_needed / ((n/4) * math.sin(2 * math.pi / n)))
    
    # Generate points for the outer polygon
    angles = np.linspace(0, 2 * np.pi, NUM_OF_POINTS, endpoint=False)
    
    utm_points = []
    for angle in angles:
        # Start with perfectly regular distance (radius)
        base_distance = regular_radius
        
        # Add randomness based on the uniformity factor
        if UNIFORM_FACTOR < 1.0:
            # The lower the uniformity factor, the more randomness
            random_variation = (1.0 - UNIFORM_FACTOR) * random.uniform(0.5, 1.5)
            distance = base_distance * (1.0 + (1.0 - UNIFORM_FACTOR) * (random_variation - 1.0))
        else:
            distance = base_distance
        
        # Calculate point coordinates
        x = center_x + distance * np.cos(angle)
        y = center_y + distance * np.sin(angle)
        utm_points.append((x, y))
    
    # Create the outer polygon in UTM
    outer_utm_polygon = Polygon(utm_points)
    
    # Calculate the area of the generated polygon
    initial_area_m2 = outer_utm_polygon.area
    
    # Scale the polygon to match the target total area
    scaling_factor = math.sqrt(total_area_needed / initial_area_m2)
    
    # Apply scaling: translate to origin, scale, translate back
    def scale_around_center(x, y):
        dx = x - center_x
        dy = y - center_y
        return center_x + dx * scaling_factor, center_y + dy * scaling_factor
    
    scaled_outer_utm_polygon = transform(lambda x, y: scale_around_center(x, y), outer_utm_polygon)
    
    # Create the holes
    holes = []
    if NUMBER_OF_DISJOINT_HOLES > 0:
        # Determine size of each hole
        hole_total_area = total_area_needed * hole_area_percentage
        area_per_hole = hole_total_area / NUMBER_OF_DISJOINT_HOLES
        
        # Calculate the maximum radius for holes to maintain disjointness
        max_hole_radius = regular_radius * scaling_factor * 0.2  # Limit to 40% of polygon radius
        
        # Calculate radius for each hole based on area
        hole_points = max(5, NUM_OF_POINTS // 2)  # Use at least 5 points for each hole
        hole_radius = min(
            math.sqrt(area_per_hole / ((hole_points/4) * math.sin(2 * math.pi / hole_points))),
            max_hole_radius
        )
        
        # Calculate positions for the holes
        # For simplicity, we'll place them in a circle around the center
        if NUMBER_OF_DISJOINT_HOLES == 1:
            # Single hole in the center
            hole_centers = [(center_x, center_y)]
        else:
            # Multiple holes arranged in a circle
            hole_placement_radius = regular_radius * scaling_factor * 0.5  # Place at 50% of the way to edge
            hole_angles = np.linspace(0, 2 * np.pi, NUMBER_OF_DISJOINT_HOLES, endpoint=False)
            hole_centers = [
                (
                    center_x + hole_placement_radius * np.cos(angle),
                    center_y + hole_placement_radius * np.sin(angle)
                )
                for angle in hole_angles
            ]
        
        # Create each hole
        for i, (hcx, hcy) in enumerate(hole_centers):
            hole_angles = np.linspace(0, 2 * np.pi, hole_points, endpoint=False)
            hole_points_coords = []
            
            # Generate points for this hole
            for angle in hole_angles:
                # Start with perfectly regular distance
                base_distance = hole_radius
                
                # Add randomness based on the uniformity factor
                if UNIFORM_FACTOR < 1.0:
                    random_variation = (1.0 - UNIFORM_FACTOR) * random.uniform(0.7, 1.3)
                    distance = base_distance * (1.0 + (1.0 - UNIFORM_FACTOR) * (random_variation - 1.0))
                else:
                    distance = base_distance
                
                # Calculate point coordinates
                x = hcx + distance * np.cos(angle)
                y = hcy + distance * np.sin(angle)
                hole_points_coords.append((x, y))
            
            # Create the hole polygon
            hole = Polygon(hole_points_coords)
            
            # Ensure the hole is contained within the outer polygon
            if hole.within(scaled_outer_utm_polygon):
                holes.append(hole)
    
    # Create the final polygon with holes
    if holes:
        polygon_with_holes = Polygon(scaled_outer_utm_polygon.exterior.coords, 
                                    [h.exterior.coords for h in holes])
    else:
        polygon_with_holes = scaled_outer_utm_polygon
    
    # Verify the final area to ensure it matches the target
    final_area_m2 = polygon_with_holes.area
    
    # Fine-tune scaling to get exact area if needed
    fine_tune_factor = math.sqrt(target_area_m2 / final_area_m2)
    
    def fine_tune_scale(x, y):
        dx = x - center_x
        dy = y - center_y
        return center_x + dx * fine_tune_factor, center_y + dy * fine_tune_factor
    
    final_utm_polygon = transform(lambda x, y: fine_tune_scale(x, y), polygon_with_holes)
    
    # Transform the UTM polygon back to WGS84 (EPSG:4326)
    wgs84_polygon = transform(transformer_from_utm.transform, final_utm_polygon)
    
    # Return the WKT representation
    return wgs84_polygon.wkt

In [12]:
NUM_OF_POINTS = 30
AREA_IN_km = 5.0
UNIFORM_FACTOR = 0.4
NUMBER_OF_DISJOINT_HOLES = 1
for POINTS in range(3, NUM_OF_POINTS, 1):
    polygon = genPolygonWithHoles(INPUT_BBOX, POINTS, AREA_IN_km, UNIFORM_FACTOR, NUMBER_OF_DISJOINT_HOLES)
    # print(f"Polygon Points: {POINTS} \n\n{polygon}\n\n")
    displayOSM(POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

KeyboardInterrupt: 

## BM 4 - POLYGON WITH TWO HOLES WITH AREA AND VARYING NUMBER OF POINTS

In [13]:
NUM_OF_POINTS = 50
AREA_IN_km = 100.0
UNIFORM_FACTOR = 0.4
NUMBER_OF_DISJOINT_HOLES = 2
  
for POINTS in range(3, NUM_OF_POINTS, 10):
    polygon = genPolygonWithHoles(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR, NUMBER_OF_DISJOINT_HOLES)
    print(f"Polygon Points: {POINTS} \n\n{polygon}\n\n")
    displayOSM(POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

Polygon Points: 43 

POLYGON ((11.614218481957497 48.15338131673646, 11.632481870588858 48.16056520887552, 11.616685352695923 48.16601250092055, 11.59849326049619 48.16845136509775, 11.59976367067751 48.17449383088427, 11.59990987861444 48.18108333175801, 11.605377322578805 48.19205187851333, 11.59777410824574 48.19692919058208, 11.593651650742334 48.2053384724686, 11.575082827497353 48.19786241597084, 11.567792840761959 48.20212891277394, 11.55856115120518 48.203321993330256, 11.550515133220475 48.21223812730127, 11.539771100580682 48.21124155231166, 11.532478695360314 48.19679919483561, 11.518106774653154 48.210943568273365, 11.507119812813736 48.20954079843504, 11.504704886642468 48.19762406020396, 11.496756467599853 48.194357394821246, 11.491635608409167 48.18895904963945, 11.490646202605241 48.18185888469421, 11.46694096990294 48.184877818261235, 11.462689902506884 48.178107070416516, 11.465165533414 48.170048236003105, 11.455934015845942 48.164210075666496, 11.466043932520604 48.

In [14]:
NUMBER_OF_DISJOINT_HOLES = 5
  
for POINTS in range(3, NUM_OF_POINTS, 10):
    polygon = genPolygonWithHoles(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR, NUMBER_OF_DISJOINT_HOLES)
    print(f"Polygon Points: {POINTS} \n\n{polygon}\n\n")
    displayOSM(POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

Polygon Points: 43 

POLYGON ((11.623230613214258 48.15317585400276, 11.638027601702552 48.160905230679795, 11.607725602883376 48.16468882630597, 11.621833007026984 48.17402730486817, 11.621179299367652 48.18174590687953, 11.59922047577409 48.180771617133, 11.59373478697922 48.185221682815566, 11.596298640586166 48.19581562895292, 11.595723832191137 48.207366343255266, 11.572380307028022 48.1943344257449, 11.572879902213762 48.21151372945075, 11.55830503452271 48.20256275730783, 11.550017476878478 48.20878493401226, 11.540204412773178 48.20151534115404, 11.527566884239404 48.21773389696838, 11.521893159922596 48.20218664191338, 11.511946699437244 48.20205909701936, 11.50886443123351 48.19290861496385, 11.488867443354223 48.20117136400492, 11.491239151339348 48.18922468188757, 11.460586225537494 48.19747576780002, 11.46750958397453 48.184652587639185, 11.463408379282827 48.17789876800748, 11.44405823535203 48.17415795431052, 11.44384906701615 48.16549430368283, 11.473802628712239 48.156

## BM 5

# Storing Polygon 

In [35]:
random.seed(777)

## varying num of points (area fixed)

In [47]:
NUM_OF_POINTS = 10000
AREA_IN_km = 5 
UNIFORM_FACTOR = 0.5

df = pd.DataFrame(columns=['points', 'geometry'])

for POINTS in range(3, NUM_OF_POINTS, 1):
    polygon = genPolygonExp(INPUT_BBOX, POINTS, AREA_IN_km, UNIFORM_FACTOR)
    new_row = pd.DataFrame({'points': [POINTS], 'geometry': [polygon]})
    df = pd.concat([df, new_row], ignore_index=True)

df

,points,geometry
0,3,"POLYGON ((11.56847998245503 48.15441316592536,..."
1,4,POLYGON ((11.564324326943115 48.15450601250767...
2,5,POLYGON ((11.561026443922522 48.15457958738135...
3,6,POLYGON ((11.563135014562553 48.15453255667647...
4,7,POLYGON ((11.559790233659225 48.15460714246011...
...,...,...
9992,9995,POLYGON ((11.558947197773053 48.15462592605537...
9993,9996,POLYGON ((11.559742212883663 48.15460821257247...
9994,9997,POLYGON ((11.556569610597496 48.15467886744611...
9995,9998,POLYGON ((11.557549323860972 48.15465705828271...


In [74]:
k =  1000
displayOSM(df['points'][k], df['geometry'][k], INPUT_BBOX)

In [10]:
df.to_csv("10000points_250km_uf5.csv", index=False)

In [49]:
df.to_pickle("10000points_5km_uf5.pkl")

In [12]:
df['geometry_wkt'] = df['geometry'].apply(lambda x: wkt.loads(x))    

In [13]:
df

,points,geometry,geometry_wkt
0,3,POLYGON ((11.648457351800591 48.15259695711923...,POLYGON ((11.648457351800591 48.15259695711923...
1,4,POLYGON ((11.631386557967762 48.15298930049101...,POLYGON ((11.631386557967762 48.15298930049101...
2,5,"POLYGON ((11.628089590838838 48.1530647829155,...","POLYGON ((11.628089590838838 48.1530647829155,..."
3,6,POLYGON ((11.633634447445827 48.15293778183169...,POLYGON ((11.633634447445827 48.15293778183169...
4,7,POLYGON ((11.626057822063448 48.15311125204565...,POLYGON ((11.626057822063448 48.15311125204565...
...,...,...,...
9992,9995,POLYGON ((11.620231231958769 48.15324431391525...,POLYGON ((11.620231231958769 48.15324431391525...
9993,9996,POLYGON ((11.625354955129275 48.15312731912376...,POLYGON ((11.625354955129275 48.15312731912376...
9994,9997,"POLYGON ((11.626916193726 48.15309162438809, 1...","POLYGON ((11.626916193726 48.15309162438809, 1..."
9995,9998,POLYGON ((11.607667229761262 48.15353023066115...,POLYGON ((11.607667229761262 48.15353023066115...


In [14]:
type(df['geometry_wkt'][3])

shapely.geometry.polygon.Polygon

In [15]:
type(df['geometry'][3])

str

In [43]:
df_drive = pd.read_pickle('10000points_50km_uf5.pkl')
df_drive

,points,geometry
0,3,POLYGON ((11.628514861494592 48.15305505187997...
1,4,"POLYGON ((11.604572022102897 48.1536004564228,..."
2,5,"POLYGON ((11.5972000147664 48.153767380116754,..."
3,6,"POLYGON ((11.603284978426924 48.1536296329816,..."
4,7,POLYGON ((11.595978201865035 48.15379499973601...
...,...,...
9992,9995,POLYGON ((11.595718609878997 48.15380086625064...
9993,9996,POLYGON ((11.602381699755563 48.15365010117676...
9994,9997,POLYGON ((11.596076400563373 48.15379278039221...
9995,9998,POLYGON ((11.59113339279515 48.153904390636264...


In [45]:
k = 100
displayOSM(df_drive['points'][k], df_drive['geometry'][k], INPUT_BBOX)

## AREA varying

In [100]:
del df

In [111]:
NUM_OF_POINTS = 10000
UNIFORM_FACTOR = 0.5

df = pd.DataFrame(columns=['area', 'geometry'])

for AREA_IN_km in np.arange(5, calculateArea(INPUT_BBOX_WKT_STRING), 5):
    polygon = genPolygonExp(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR)
    new_row = pd.DataFrame({'area': [AREA_IN_km], 'geometry': [polygon]})
    df = pd.concat([df, new_row], ignore_index=True)

df

/tmp/ipykernel_48/2049779023.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


,area,geometry
0,5.0,POLYGON ((11.55879141818979 48.154629396285806...
1,10.0,POLYGON ((11.566116596303248 48.15446598769269...
2,15.0,"POLYGON ((11.57258995506804 48.15432119189535,..."
3,20.0,POLYGON ((11.575671810881422 48.15425212869934...
4,25.0,POLYGON ((11.578361089322778 48.15419179534623...
...,...,...
106,535.0,POLYGON ((11.716181965681184 48.15101538013420...
107,540.0,POLYGON ((11.744897595433462 48.15033270576317...
108,545.0,POLYGON ((11.728961350641688 48.15071245608239...
109,550.0,POLYGON ((11.70030056245125 48.151389849828455...


In [112]:
k =  80
displayOSM(df['area'][k], df['geometry'][k], INPUT_BBOX)

In [113]:
df.to_pickle("10000points_km_uf5.pkl")

## varying num of points (area fixed) with single hole

In [26]:
import gc

In [41]:
del df
NUM_OF_POINTS = 10000
AREA_IN_km = 500.0
UNIFORM_FACTOR = 0.5
NUMBER_OF_DISJOINT_HOLES = 1

df = pd.DataFrame(columns=['points', 'geometry'])

for POINTS in tqdm(range(3, NUM_OF_POINTS, 1)):
    polygon = genPolygonWithHoles(INPUT_BBOX, POINTS, AREA_IN_km, UNIFORM_FACTOR, NUMBER_OF_DISJOINT_HOLES)
    new_row = pd.DataFrame({'points': [POINTS], 'geometry': [polygon]})
    df = pd.concat([df, new_row], ignore_index=True)
    del polygon
    gc.collect()


df

  0%|          | 0/9997 [00:00<?, ?it/s]

,points,geometry
0,3,POLYGON ((11.822725334528279 48.14844629497073...
1,4,"POLYGON ((11.750771384885711 48.1501921784989,..."
2,5,POLYGON ((11.752238758743836 48.15015702538612...
3,6,POLYGON ((11.709750620483145 48.15116729070517...
4,7,POLYGON ((11.740972987677667 48.15042643219243...
...,...,...
9992,9995,POLYGON ((11.724237597693062 48.15082459442772...
9993,9996,POLYGON ((11.72287129970581 48.150856993028256...
9994,9997,"POLYGON ((11.7362568005127 48.15053888518627, ..."
9995,9998,"POLYGON ((11.71688707641175 48.15099870323387,..."


In [43]:
k =  4000
displayOSM(df['points'][k], df['geometry'][k], INPUT_BBOX)

In [44]:
df.to_pickle("10000points_500km_uf5_1hole.pkl")

# Sweden

In [4]:
INPUT_BBOX = [11.015629, 55.128649, 24.199222, 69.380313] ## Sweden
INPUT_BBOX_WKT_STRING = (box(*INPUT_BBOX, ccw=True)).wkt
INPUT_BBOX_WKT_STRING

'POLYGON ((24.199222 55.128649, 24.199222 69.380313, 11.015629 69.380313, 11.015629 55.128649, 24.199222 55.128649))'

In [5]:
calculateArea(INPUT_BBOX_WKT_STRING) 

1073940.88

In [42]:
def displayOSM(i, polygon, bbox, area_percentage=None, uniform=None):
    # Create a map with the center based on the bounding box
    map_center = [(bbox[1] + bbox[3]) / 2, (bbox[0] + bbox[2]) / 2]
    m = folium.Map(location=map_center, zoom_start=4) # SWEDEN ZOOM 4
    
    if isinstance(polygon, str):
        # Assuming the string is WKT format
        try:
            shape = wkt.loads(polygon)
            geo_json = shape.__geo_interface__
        except Exception as e:
            print(f"Error parsing polygon {e}")
    else:
        # Assuming it's already in a format folium can use
        geo_json = polygon
        
        # Add the polygon to the map with a unique style
    folium.GeoJson(
        geo_json,
        style_function=lambda x, color=f'#4400ff': {
                'fillColor': color,
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5',
                'fillOpacity': 0.5,
        }
    ).add_to(m)

    north, south, east, west = bbox[3], bbox[1], bbox[0], bbox[2]
    # Add the bounding box to the map
    folium.Rectangle(bounds=[[north, west], [south, east]], color="red", fill=False).add_to(m)

    folium.map.Marker(
        [north-0.25, east-0.1],
        icon=DivIcon(
            icon_size=(550,56),
            icon_anchor=(0,0),
            html='<div style="font-size: 18pt; color: #4400ff;">points '+str(i)+' | area '+str(area_percentage)+' km² | uniformity '+str(uniform)+'</div>',
            )
        ).add_to(m)
    display(m)
    clear_output(wait=True)
    time.sleep(0.5)  # Pause to see the update
    # return m

def genPolygonExp(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR):
    """
    Generate a polygon with specified area in square kilometers
    
    Parameters:
    INPUT_BBOX: WKT string representing the bounding box
    NUM_OF_POINTS: Number of points in the polygon
    AREA_IN_km²: Target area in square kilometers
    UNIFORM_FACTOR: Controls shape regularity (0.0: very irregular, 1.0: perfectly regular)
    
    Returns:
    WKT string representing the generated polygon in EPSG:4326
    """
    # Parse the bounding box
    if type(INPUT_BBOX) == str:
        bbox = wkt.loads(INPUT_BBOX)
        bbox_bounds = bbox.bounds
        min_x, min_y, max_x, max_y = bbox_bounds
    else:
        min_x, min_y, max_x, max_y = INPUT_BBOX
        bbox = wkt.loads((box(*INPUT_BBOX, ccw=True)).wkt)
    
    # Compute the centroid for UTM zone determination
    centroid_x = (min_x + max_x) / 2
    centroid_y = (min_y + max_y) / 2
    
    # Determine UTM zone
    utm_zone = int((centroid_x + 180) / 6) + 1
    proj_string = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    # Create transformers for WGS84 <-> UTM conversion
    transformer_to_utm = Transformer.from_crs("EPSG:4326", proj_string, always_xy=True)
    transformer_from_utm = Transformer.from_crs(proj_string, "EPSG:4326", always_xy=True)
    
    # Transform the bounding box to UTM
    utm_bbox = transform(transformer_to_utm.transform, bbox)
    utm_bounds = utm_bbox.bounds
    utm_min_x, utm_min_y, utm_max_x, utm_max_y = utm_bounds
    
    # Calculate the center point of the bounding box in UTM
    center_x = (utm_min_x + utm_max_x) / 2
    center_y = (utm_min_y + utm_max_y) / 2
    
    # Calculate the radius needed for the target area
    # For a regular polygon with n sides, area = (n/4) * r² * sin(2π/n)
    # Solving for r: r = sqrt(area / ((n/4) * sin(2π/n)))
    target_area_m2 = AREA_IN_km * 1_000_000  # Convert km² to m²
    
    # For a regular polygon, calculate the radius
    n = NUM_OF_POINTS
    regular_radius = math.sqrt(target_area_m2 / ((n/4) * math.sin(2 * math.pi / n)))
    
    # Generate points for the polygon
    angles = np.linspace(0, 2 * np.pi, NUM_OF_POINTS, endpoint=False)
    
    utm_points = []
    for angle in angles:
        # Start with perfectly regular distance (radius)
        base_distance = regular_radius
        
        # Add randomness based on the uniformity factor
        if UNIFORM_FACTOR < 1.0:
            # The lower the uniformity factor, the more randomness
            random_variation = (1.0 - UNIFORM_FACTOR) * random.uniform(0.5, 1.5)
            distance = base_distance * (1.0 + (1.0 - UNIFORM_FACTOR) * (random_variation - 1.0))
        else:
            distance = base_distance
        
        # Calculate point coordinates
        x = center_x + distance * np.cos(angle)
        y = center_y + distance * np.sin(angle)
        utm_points.append((x, y))
    
    # Create the polygon in UTM
    utm_polygon = Polygon(utm_points)
    
    # Calculate the area of the generated polygon
    initial_area_m2 = utm_polygon.area
    
    # Scale the polygon to match the target area
    scaling_factor = math.sqrt(target_area_m2 / initial_area_m2)
    
    # Apply scaling: translate to origin, scale, translate back
    def scale_around_center(x, y):
        dx = x - center_x
        dy = y - center_y
        return center_x + dx * scaling_factor, center_y + dy * scaling_factor
    
    scaled_utm_polygon = transform(lambda x, y: scale_around_center(x, y), utm_polygon)
    
    # Transform the UTM polygon back to WGS84 (EPSG:4326)
    wgs84_polygon = transform(transformer_from_utm.transform, scaled_utm_polygon)
    
    # Return the WKT representation
    return wgs84_polygon.wkt

In [43]:
NUM_OF_POINTS = 20
AREA_IN_km = 100
UNIFORM_FACTOR = 0.5

for POINTS in range(3, NUM_OF_POINTS, 5):
    polygon = genPolygonExp(INPUT_BBOX, POINTS, AREA_IN_km, UNIFORM_FACTOR)
    displayOSM(POINTS, polygon, INPUT_BBOX, AREA_IN_km, UNIFORM_FACTOR)

In [52]:
NUM_OF_POINTS = 100
UNIFORM_FACTOR = 0.5

df = pd.DataFrame(columns=['area', 'geometry'])

for AREA_IN_km in  np.arange(5, calculateArea(INPUT_BBOX_WKT_STRING), 100000):
    polygon = genPolygonExp(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR)
    new_row = pd.DataFrame({'area': [AREA_IN_km], 'geometry': [polygon]})
    df = pd.concat([df, new_row], ignore_index=True)

df

/tmp/ipykernel_1258/2285776573.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


,area,geometry
0,5.0,POLYGON ((18.231915409867433 62.37503530732262...
1,100005.0,POLYGON ((21.388226987163335 62.26558872019588...
2,200005.0,"POLYGON ((23.3361683408742 62.161624822198156,..."
3,300005.0,"POLYGON ((24.83354164914195 62.06248491680234,..."
4,400005.0,POLYGON ((24.449449413929024 62.08952423426346...
5,500005.0,POLYGON ((26.470449878481087 61.93466005528702...
6,600005.0,POLYGON ((26.100537711860227 61.96534127982474...
7,700005.0,"POLYGON ((25.82562865091264 61.98746111839856,..."
8,800005.0,POLYGON ((26.986316219802838 61.89010745119861...
9,900005.0,POLYGON ((29.875103310001883 61.60195402669811...


In [53]:
k =  7
displayOSM(df['area'][k], df['geometry'][k], INPUT_BBOX)

In [56]:
NUM_OF_POINTS = 100
UNIFORM_FACTOR = 0.5

df = pd.DataFrame(columns=['area', 'geometry'])

for AREA_IN_km in np.arange(5, calculateArea(INPUT_BBOX_WKT_STRING), 105):
    polygon = genPolygonExp(INPUT_BBOX, NUM_OF_POINTS, AREA_IN_km, UNIFORM_FACTOR)
    new_row = pd.DataFrame({'area': [AREA_IN_km], 'geometry': [polygon]})
    df = pd.concat([df, new_row], ignore_index=True)

df

/tmp/ipykernel_1258/1027044102.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_row], ignore_index=True)


,area,geometry
0,5.0,POLYGON ((18.229713843942655 62.37508643612464...
1,110.0,POLYGON ((18.335425667895933 62.37259191413835...
2,215.0,POLYGON ((18.388918687157254 62.37129887952272...
3,320.0,"POLYGON ((18.39052601018052 62.37125970753531,..."
4,425.0,POLYGON ((18.395812881966922 62.37113072985505...
...,...,...
10223,1073420.0,POLYGON ((30.875929500905126 61.48648086361173...
10224,1073525.0,POLYGON ((30.012652840248254 61.58656860653023...
10225,1073630.0,POLYGON ((28.557244629326412 61.74162869054392...
10226,1073735.0,"POLYGON ((30.49932407332527 61.53089507337773,..."


In [57]:
df.to_pickle("100points_uf5_SWEDEN.pkl")